In [ ]:
import pandas as pd
import random

# Define board spaces
luck_spaces = [3, 7, 13, 17, 23, 27, 33, 37, 43, 47, 53, 57]
credit_spaces = [5, 15, 25, 35, 45, 55]
roll_again_spaces = [10, 30, 50]
center_path_spaces = [20, 40, 60]
final_center_path = list(range(61, 71))  # Final 10-space center path

# Define a list of Luck Card effects
luck_cards = [
    "Powerful: Advance to Start of any 3 Paths, If Already There, Advance to Infinity",
    "Banished: Go to Odds Lock",
    "Banished: Go to Evens Lock",
    "Take Another Turn (It's Your Lucky Day)",
    "Superstar! Take Another Turn for Sharing Your Skills",
    "Awesomeness! Take Another Turn to Show Your Brilliant Educational Skills",
    "Caught in the Act! Lose a Turn",
    "Wrong Change! Lose 2 Turns",
    "Oops! Lose School Bag, Pay 50 Credits or Lose 2 Turns",
    "Move 3 Spaces Forward",
    "Move 3 Spaces Forward",
    "Move 5 Spaces Forward",
    "Move 5 Spaces Forward",
    "Move 7 Spaces Forward",
    "Move 7 Spaces Forward",
    "Winner! Receive 2 Money Tokens",
    "Good Citizen! Receive 1 Money Token for Charity Work",
    "Eureka! Win Math Quiz, Collect 2 Money Tokens",
    "Games Arcade! Receive Token from Cash",
    "It's ShowTime! Reveal Money Tokens, Give Back a Token if Over 50 Credits",
    "Let It Go! Give Credit Tokens Back to the Cash Vault"
]

# Create a full board with position types
board = []
for i in range(71):
    if i in luck_spaces:
        tile_type = "Luck_Space"
    elif i in credit_spaces:
        tile_type = "Credit_Space"
    elif i in roll_again_spaces:
        tile_type = "Roll_Again_Space"
    elif i in center_path_spaces:
        tile_type = "Center_Path"
    elif i in final_center_path:
        tile_type = "Final_Center_Path"
    else:
        tile_type = "Neutral_Space"
    board.append({"Position": i, "Type": tile_type})


def get_position_type(position):
    if 0 <= position < len(board):
        return board[position]["Type"]
    return "Unknown"


def simulate_game(player_id, opponent_id):
    credit_notes = [10]*20 + [20]*14 + [50]*14
    random.shuffle(credit_notes)

    credits = 0
    position = 0
    new_position = position
    heading_to_center = False
    skip_turns = 0
    take_extra_turn = False
    data = []

    while True:
        if skip_turns > 0:
            skip_turns -= 1
            data.append({
                "Player_ID": player_id,
                "Dice_Roll_1": None,
                "Dice_Roll_2": None,
                "Math_Operation": None,
                "Move_Direction": None,
                "Move_Value": 0,
                "Resulting_Position": position,
                "Position_Type": get_position_type(position),
                "Credits_Before": credits,
                "Credits_After": credits,
                "Luck_Card_Used": "Skipped Turn",
                "Opponent_Interaction": "None",
                "Decision_Taken": "Skipped Turn due to penalty",
                "Won_Game": 0
            })
            continue

        # Roll dice and determine operation
        dice1 = random.randint(1, 12)
        dice2 = random.randint(1, 12)
        operation = random.choice(["Add", "Subtract", "Multiply", "Divide"])

        if operation == "Add":
            move = dice1 + dice2
        elif operation == "Subtract":
            move = abs(dice1 - dice2)
        elif operation == "Multiply":
            move = dice1 * dice2
        else:  # Divide
            larger, smaller = max(dice1, dice2), min(dice1, dice2)
            if smaller != 0 and larger % smaller == 0:
                move = larger // smaller
            else:
                move = larger - smaller
                operation = "Subtract"

        direction = random.choices(["Forward", "Backward"], weights=[0.8, 0.2])[0]
        new_position = position + move if direction == "Forward" else max(0, position - move)

        if not heading_to_center:
            new_position %= 61
        else:
            new_position = min(new_position, 70)

        if credits >= 200 and not heading_to_center:
            heading_to_center = True

        if heading_to_center and new_position in center_path_spaces:
            new_position += 1

        pos_type = get_position_type(new_position)
        prev_credits = credits
        card_used = "No"
        decision = f"Moved {direction} using {operation} → {move}"
        won_game = 1 if new_position in final_center_path else 0

        # Space effects
        if pos_type == "Credit_Space":
            if credit_notes:
                payout = credit_notes.pop()
                credits += payout
                decision += f"; Gained {payout} Credits"
            else:
                decision += "; No credits left in vault"
        elif pos_type == "Luck_Space" and luck_cards:
            card = random.choice(luck_cards)
            card_used = card
            decision += f"; Luck Card: {card}"
            # Card effects
            if "Lose 2 Turns" in card:
                skip_turns += 2
            elif "Lose a Turn" in card:
                skip_turns += 1
            elif "Take Another Turn" in card:
                take_extra_turn = True
            elif "Move 3 Spaces Forward" in card:
                new_position = min(new_position + 3, 70)
            elif "Move 5 Spaces Forward" in card:
                new_position = min(new_position + 5, 70)
            elif "Move 7 Spaces Forward" in card:
                new_position = min(new_position + 7, 70)
            elif "Pay 50 Credits" in card and credits >= 50:
                credits -= 50
                decision += "; Paid 50 credits"
            elif "Give Credit Tokens Back" in card:
                refund = min(credits, random.choice([10, 20, 50]))
                credits -= refund
                decision += f"; Gave back {refund} credits"

        # Check again if won after any luck effect movement
        if heading_to_center and new_position in final_center_path:
            won_game = 1

        data.append({
            "Player_ID": player_id,
            "Dice_Roll_1": dice1,
            "Dice_Roll_2": dice2,
            "Math_Operation": operation,
            "Move_Direction": direction,
            "Move_Value": move,
            "Resulting_Position": new_position,
            "Position_Type": pos_type,
            "Credits_Before": prev_credits,
            "Credits_After": credits,
            "Luck_Card_Used": card_used,
            "Opponent_Interaction": "None",
            "Decision_Taken": decision,
            "Won_Game": won_game
        })

        if won_game == 1:
            break

        position = new_position

        # Extra turn logic
        if take_extra_turn:
            take_extra_turn = False
            continue

    return data


# Main simulation loop
all_game_data = []
for game_id in range(10000):
    for player_id in [1, 2]:
        opponent_id = 2 if player_id == 1 else 1
        game_data = simulate_game(player_id, opponent_id)
        all_game_data.extend(game_data)

    if game_id % 500 == 0:
        pd.DataFrame(all_game_data).to_csv("new_game_data2.csv", index=False)

# Final save
pd.DataFrame(all_game_data).to_csv("new_game_data2.csv", index=False)

# Summary
df = pd.DataFrame(all_game_data)
print("Win Percentage:", df["Won_Game"].mean() * 100)
